## Mount the Drive, and Change to Google Drive Folder

In [10]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount = True)

# %cd /content/drive/MyDrive/MSc.-Dissertations/1/Files
%ls

 Volume in drive C has no label.
 Volume Serial Number is E858-006A

 Directory of c:\Users\Robin\Desktop\MSc.-Dissertations-1\1\Files

01/06/2023  21:30    <DIR>          .
01/06/2023  21:30    <DIR>          ..
01/06/2023  21:30            28,474 classification.ipynb
01/06/2023  21:30           983,970 flist.txt
01/06/2023  21:30         4,693,058 properties.csv
01/06/2023  21:30             4,527 randomsample.ipynb
01/06/2023  21:30            21,438 Robin.ipynb
01/06/2023  21:30    <DIR>          street_view
               5 File(s)      5,731,467 bytes
               3 Dir(s)  395,401,027,584 bytes free


## Import Libraries

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import random
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

## Count the Number of Files, and Take Random Samples from the Image Files

In [12]:
# !ls street_view
# count how many files and write the filenames into a file
# !ls street_view -1 | wc -l 
# !ls street_view/*.jpg > flist.txt
flist = list(pd.read_csv('flist.txt', header = None)[0])

# Set seed so sample is reproducible 
random.seed(99)  # set this to an integer value!!!
nsamp = 100
flist_sub = random.sample(flist, nsamp)
flist = flist_sub

# print(flist)

## Overview of the `properties` Dataset

In [13]:
properties = pd.read_csv('properties.csv')
properties.head()

,Unnamed: 0,address,propertyType,bedrooms,detailUrl,location_lat,location_lng,property_id
0,0,"12, Gorsey Brigg, Dronfield Woodhouse, Dronfie...",Terraced,3.0,https://www.rightmove.co.uk/house-prices/detai...,53.29986,-1.49446,60d9dd15-c5a0-4d9c-a341-a1d47add49d5
1,0,"5, Highgate Lane, Dronfield, Derbyshire S18 1UB",Detached,4.0,https://www.rightmove.co.uk/house-prices/detai...,53.29135,-1.45975,4a586e80-181a-4b82-b5c3-2d789436bb14
2,0,"125, Gosforth Lane, Dronfield, Derbyshire S18 1RB",Detached,3.0,https://www.rightmove.co.uk/house-prices/detai...,53.29763,-1.47573,93680b6c-237e-44d3-8f40-959a14b80cad
3,0,"80, Shakespeare Crescent, Dronfield, Derbyshir...",Detached,3.0,https://www.rightmove.co.uk/house-prices/detai...,53.29259,-1.45644,5d49758b-f148-4d06-bbae-3eb23f5c68fb
4,0,"21, Gainsborough Road, Dronfield, Derbyshire S...",Detached,NaN,https://www.rightmove.co.uk/house-prices/detai...,53.29740,-1.48503,4645f5eb-de7c-474f-8d7e-b59fa8c55f19


In [14]:
properties.propertyType = properties.propertyType.astype('category')
properties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17550 entries, 0 to 17549
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Unnamed: 0    17550 non-null  int64   
 1   address       17550 non-null  object  
 2   propertyType  17550 non-null  category
 3   bedrooms      11505 non-null  float64 
 4   detailUrl     17550 non-null  object  
 5   location_lat  17550 non-null  float64 
 6   location_lng  17550 non-null  float64 
 7   property_id   17550 non-null  object  
dtypes: category(1), float64(3), int64(1), object(3)
memory usage: 977.2+ KB


In [15]:
properties.describe()

,Unnamed: 0,bedrooms,location_lat,location_lng
count,17550.0,11505.000000,17550.000000,17550.000000
mean,0.0,2.871186,52.912264,-2.330492
std,0.0,1.010339,1.833830,1.262468
min,0.0,0.000000,50.617080,-4.268950
25%,0.0,2.000000,51.232830,-3.067290
50%,0.0,3.000000,53.095885,-2.658955
75%,0.0,3.000000,53.846760,-1.712750
max,0.0,6.000000,55.910540,0.719990


In [16]:
properties.propertyType.value_counts()

Detached         4134
Semi-Detached    4056
Unknown          3900
Terraced         3666
Flat             1794
Name: propertyType, dtype: int64

## A Subsample of the `properties` Dataset

In [17]:
flist_id = list(map(lambda string: string[16 : -4], flist))
properties_sub = pd.DataFrame(properties.loc[properties['property_id'].isin(flist_id)])
Img_mat = np.array(list(map(lambda x: np.array(Image.open(x)).reshape(-1), flist))) / 255
Img_list = list(Img_mat)
properties_sub = properties_sub.assign(Images = Img_list)
properties_sub.head()

,Unnamed: 0,address,propertyType,bedrooms,detailUrl,location_lat,location_lng,property_id,Images
639,0,"33, Palmer Crescent, Dronfield, Derbyshire S18...",Terraced,3.0,https://www.rightmove.co.uk/house-prices/detai...,53.29775,-1.46342,03585262-92c9-4d1b-8179-75115db154bd,"[0.7450980392156863, 0.7647058823529411, 0.780..."
657,0,"54, Barnard Avenue, Coal Aston, Dronfield, Der...",Semi-Detached,3.0,https://www.rightmove.co.uk/house-prices/detai...,53.31054,-1.44990,3e7f2368-90ae-4224-9398-be087ca5a347,"[0.8, 0.8549019607843137, 0.8901960784313725, ..."
771,0,"22, Loundes Road, Unstone, Dronfield, Derbyshi...",Semi-Detached,NaN,https://www.rightmove.co.uk/house-prices/detai...,53.28516,-1.44318,55786afb-835b-4898-a195-7e788646163e,"[0.5372549019607843, 0.5450980392156862, 0.698..."
795,0,"152, Eckington Road, Coal Aston, Dronfield, De...",Detached,4.0,https://www.rightmove.co.uk/house-prices/detai...,53.31108,-1.44900,0445e214-3629-4fe6-aa69-539b5f1c0130,"[0.6980392156862745, 0.796078431372549, 0.9137..."
816,0,"18, Longcroft Avenue, Dronfield Woodhouse, Dro...",Detached,4.0,https://www.rightmove.co.uk/house-prices/detai...,53.30494,-1.50520,f195df92-4ff7-430c-b64a-f37b001c6279,"[0.6705882352941176, 0.6078431372549019, 0.505..."


## PCA

In [18]:
pca = PCA()
pc_Img_mat = pca.fit_transform(Img_mat)
pc_Img_mat

array([[-1.78681223e+01, -4.75122810e+01,  1.01383432e+01, ...,
         6.92237406e-18,  5.00924378e-18,  4.79739508e-18],
       [-4.03480648e+01,  2.14626854e+00, -1.73377388e+01, ...,
         6.92237406e-18,  5.00924378e-18,  4.79739508e-18],
       [ 1.78698745e+01,  2.66806235e+01, -1.27323228e+01, ...,
         3.88860116e-17,  1.27498735e-15, -8.28081952e-17],
       ...,
       [ 1.25029215e-01,  4.04731758e+01, -4.35435176e+01, ...,
         6.92237406e-18,  5.00924378e-18,  4.79739508e-18],
       [-6.41959488e+01, -2.28378148e+01,  2.68269324e+01, ...,
         6.92237406e-18,  5.00924378e-18,  4.79739508e-18],
       [-1.64037980e+01,  3.16594960e+01,  3.83021580e+01, ...,
         6.92237406e-18,  5.00924378e-18,  4.79739508e-18]])